In [1]:
!pip install -q ../input/monai/monai-0.9.1-202207251608-py3-none-any.whl

!pip install -q ../input/lib-pretrainedmodels/pretrainedmodels-0.7.4-py3-none-any.whl
!pip install -q ../input/tim-0412/timm-0.4.12-py3-none-any.whl
!pip install -q ../input/lib-efficientnet-pytorch/efficientnet_pytorch-0.7.1-py3-none-any.whl
!pip install -q ../input/segmentation-models-pytorch/segmentation_models_pytorch-0.3.0-py3-none-any.whl


In [2]:
import torch
from torch import nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, random_split

import segmentation_models_pytorch as smp
# import torchsummary

import pandas as pd
import numpy as np
import random, shutil, time, os

import sklearn
import cv2
import matplotlib.pyplot as plt

import albumentations as A
import gc

import monai
# import tifffile as tiff

from torch.cuda import amp

import rasterio
from rasterio.windows import Window

import warnings
warnings.filterwarnings('ignore')

print('done')

done


In [3]:
CFG = {
    'lr':3e-4,
    'shape':(512, 512),

}
TRAIN = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
def seed_everything(seed=44):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
def clear_cache():
    torch.cuda.empty_cache()
    gc.collect()

In [5]:
BASE_DIR = '../input/hubmap-organ-segmentation'

mode = 'test'


DATA_DIR = os.path.join(BASE_DIR, f'{mode}_images')
            
df = pd.read_csv(os.path.join(BASE_DIR, f'{mode}.csv'))
df['path'] = df['id'].apply(lambda fname : os.path.join(DATA_DIR, str(fname) + '.tiff'))
organ_to_class = {
    'prostate':0,
    'spleen':1,
    'lung':2,
    'kidney':3,
    'largeintestine':4
}
df['classes'] = df['organ'].apply(lambda organ : organ_to_class[organ])
df.head(5)

,id,organ,data_source,img_height,img_width,pixel_size,tissue_thickness,path,classes
0,10078,spleen,Hubmap,2023,2023,0.4945,4,../input/hubmap-organ-segmentation/test_images...,1


In [6]:
# https://www.kaggle.com/paulorzp/rle-functions-run-length-encode-decode
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def img_read(path):
    img = tiff.imread(path)
    return img


class Dataset2D(torch.utils.data.Dataset):
    def __init__(self, df_sub, train=True):
        self.train = train
        
        self.paths = np.array(df_sub['path'])
        self.wid = np.array(df_sub['img_width'])
        self.hei = np.array(df_sub['img_height'])
        
        
    def __len__(self):
        return len(self.paths)
    
    def transform(self, img):
        trans = A.Compose([
            A.RandomScale(scale_limit=(0.1, 0.1), p=1),
            A.CenterCrop(CFG['shape'][0], CFG['shape'][1], always_apply=True, p=1.0)
        ])
        return trans(image=img)
    
    def data_prep_aug(self, img):
        shape = CFG['shape']
        img = (cv2.resize(img, shape, interpolation=cv2.INTER_AREA) / img.max()).astype('float32')
        
#         trans = self.transform(img)
#         img = trans['image']
        
        img = img.transpose(2,0,1)

        return torch.tensor(img, dtype=torch.float16, device=device)
    
    def __getitem__(self, idx):
        shape = CFG['shape']
        img = img_read(self.paths[idx])

        # data preprocessing and augmentation
        img = self.data_prep_aug(img)
        
        return img

In [7]:
MASKS = f'../input/hubmap-organ-segmentation/{mode}.csv'

# functions to convert encoding to mask and mask to encoding
def enc2mask(encs, shape):
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for m,enc in enumerate(encs):
        if isinstance(enc,np.float) and np.isnan(enc): continue
        s = enc.split()
        for i in range(len(s)//2):
            start = int(s[2*i]) - 1
            length = int(s[2*i+1])
            img[start:start+length] = 1 + m
    return img.reshape(shape).T

def mask2enc(mask, n=1):
    pixels = mask.T.flatten()
    encs = []
    for i in range(1,n+1):
        p = (pixels == i).astype(np.int8)
        if p.sum() == 0: encs.append(np.nan)
        else:
            p = np.concatenate([[0], p, [0]])
            runs = np.where(p[1:] != p[:-1])[0] + 1
            runs[1::2] -= runs[::2]
            encs.append(' '.join(str(x) for x in runs))
    return encs

# df_masks = pd.read_csv(MASKS)[['id', 'rle']].set_index('id')
# df_masks.head()

In [8]:
sz = 512   # the size of tiles
reduce = 2 # reduce the original images by X times 

s_th = 40  # saturation blancking threshold
p_th = 1000*(sz // 256) ** 2 # threshold for the minimum number of pixels

mean = np.array([0.77455656, 0.74896831, 0.76683053])
std = np.array([0.25168728, 0.2655022, 0.26106301])

DATA = DATA_DIR

class HuBMAPDataset(torch.utils.data.Dataset):
    def __init__(self, idx, sz=sz, reduce=reduce, encs=None):
        self.data = rasterio.open(os.path.join(DATA,str(idx)+'.tiff'),num_threads='all_cpus')

        if self.data.count != 3:
            subdatasets = self.data.subdatasets
            self.layers = []
            if len(subdatasets) > 0:
                for i, subdataset in enumerate(subdatasets, 0):
                    self.layers.append(rasterio.open(subdataset))
                    
        self.shape = self.data.shape
        self.reduce = reduce
        self.sz = reduce*sz
        self.pad0 = (self.sz - self.shape[0]%self.sz)%self.sz
        self.pad1 = (self.sz - self.shape[1]%self.sz)%self.sz
        self.n0max = (self.shape[0] + self.pad0)//self.sz
        self.n1max = (self.shape[1] + self.pad1)//self.sz
        self.mask = enc2mask(encs,(self.shape[1],self.shape[0])) if encs is not None else None
        
    def __len__(self):
        return self.n0max*self.n1max
    
    def __getitem__(self, idx):
        n0,n1 = idx//self.n1max, idx%self.n1max

        x0,y0 = -self.pad0//2 + n0*self.sz, -self.pad1//2 + n1*self.sz

        # make sure that the region to read is within the image
        p00,p01 = max(0,x0), min(x0 + self.sz,self.shape[0])
        p10,p11 = max(0,y0), min(y0 + self.sz,self.shape[1])

        img = np.zeros((self.sz,self.sz,3),np.uint8)
        
        # mapping the loade region to the tile
        if self.data.count == 3:
            img[(p00-x0):(p01-x0),(p10-y0):(p11-y0)] = np.moveaxis(self.data.read([1,2,3],
                window=Window.from_slices((p00,p01),(p10,p11))), 0, -1)
        else:
            for i,layer in enumerate(self.layers):
                img[(p00-x0):(p01-x0),(p10-y0):(p11-y0),i] =\
                  layer.read(1,window=Window.from_slices((p00,p01),(p10,p11)))
        
        if self.reduce != 1:
            img = cv2.resize(img,(self.sz//reduce,self.sz//reduce),
                             interpolation = cv2.INTER_AREA)

        m = 1
        if img.max() == 0:
            m = 1
        else:
            m = img.max()
        img = (img/m - mean) / std
        img = torch.tensor([img.transpose(2,0,1)], dtype=torch.float, device=device)
    
        #return -1 for empty images
        return img

In [9]:
# for index in df['id']:
#     ds = HuBMAPDataset(index)
#     for i, a in enumerate(ds):
#         print(a[2])
#         if i == len(ds) - 1:
#             break

In [10]:
# fig=plt.figure(figsize=(12, 12))
# # plt.subplots(3,3)
# for index in df['id'][:1]:
#     ds = HuBMAPDataset(index)
    
    
#     for i, a in enumerate(ds):
#         print(a[0].shape)
# #         fig.add_subplot((i+1)//3+1, i%3+1, (i+1)//3+(i+1)%3)
# #         plt.axis('off')
# #         if (i+1)//3 == 0:
# #             a[0] = a[0][]
#         plt.subplots(1)
#         plt.imshow(a[0])
#         plt.axis('off')
#         if i == 3:
#             break
#         pass
# #         print('as')
# #     break


In [11]:
# UNet EFFB7
activation = None
model = smp.Unet(
    encoder_weights=None,
    encoder_name='efficientnet-b7',
    decoder_channels= (512, 256, 128, 64, 32),
    decoder_use_batchnorm=True,
    activation=activation,
    in_channels=3,
    classes=5,
)
model = model.to(device)
model.eval()
print('evaled')

evaled


In [12]:
organ_to_class = {
    'prostate':0,
    'spleen':1,
    'lung':2,
    'kidney':3,
    'largeintestine':4
}

In [13]:
# for i, images in enumerate(ds):
#     print(images[1].shape)
    
#     if i == len(ds) - 1:
#         break

In [14]:
l = []


t = time.time()
for fold in range(5):
    model.load_state_dict(torch.load(f'../input/uneteffb7-9tiled-20-17-20-20-17/PostRUNS_fold{fold}.pt'))    
    model.eval()
    
    clear_cache()
    l_index = 0

    for rn, index in enumerate(df['id']):
        ds = HuBMAPDataset(index)
#         ds_loader = torch.utils.data.DataLoader(ds, batch_size=4)
        try:
            for i, images in enumerate(ds):
                clear_cache()

                with torch.no_grad():
                    y_pred = model(images)

                y_pred = nn.Sigmoid()(y_pred)[0][organ_to_class[df['organ'][rn]]]

    #             y_pred[y_pred < 0.5] = 0
    #             y_pred[y_pred > 0.5] = 1
    #             y_pred[y_pred == 0.5] = 1
    #             plt.subplots()
    #             plt.imshow(y_pred.detach().cpu().numpy())
    #             print(y_pred.detach().cpu().numpy().max())

                if fold == 0:
                    l.append(y_pred)
                else:
                    l[l_index] += y_pred
                    l_index += 1


                if i == len(ds) - 1:
                    break
        except:
            if fold == 0:
                l.append(np.zeros((512,512)))
            else:
                l[l_index] += y_pred
                l_index += 1

    print(f'fold {fold} done')

    
# Convert the data into 1/0
for a in range(len(l)):
    l[a] = l[a].detach().cpu().numpy()

l = np.array(l) / 5

l[l > 0.5] = 1
l[l == 0.5] = 1
l[l < 0.5] = 0

# So by now you can just re-convert to images!


print(time.time() - t)

fold 0 done
fold 1 done
fold 2 done
fold 3 done
fold 4 done
29.667675256729126


In [15]:
submission = pd.DataFrame(columns=['id', 'rle'])

for rn, row in enumerate(df.iloc):
    ds = HuBMAPDataset(int(row['id']))
    start_ind = 0
    
    tile_size = 512
    
    arr = np.zeros((tile_size*ds.n0max, tile_size*ds.n0max))
    
    try:
        for i in range(ds.n0max):
            for j in range(ds.n1max):
                arr[i*tile_size:(i+1)*tile_size, j*tile_size:(j+1)*tile_size] = l[start_ind]
                start_ind += 1
        arr = cv2.resize(arr, (int(row['img_width']), int(row['img_height'])))

        arr[arr < 0.5] = 0
        arr[arr > 0.5] = 1
        arr[arr == 0.5] = 1
    except:
        pass
    
    rle = rle_encode(arr)
    submission.loc[len(submission)] = [int(row['id']), rle]
    
#     arr = arr[ds.pad0:arr.shape[0]-ds.pad0, ds.pad1:arr.shape[1]-ds.pad1]

In [16]:
# plt.imshow(enc2mask(df[ind:ind+1]['rle'], shape=(3000,3000)))

In [17]:
df

,id,organ,data_source,img_height,img_width,pixel_size,tissue_thickness,path,classes
0,10078,spleen,Hubmap,2023,2023,0.4945,4,../input/hubmap-organ-segmentation/test_images...,1


In [18]:
submission.to_csv('submission.csv', index=False)